In [ ]:
# for plotting

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()
sns.set_context('poster')
sns.set_style('ticks')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'cmr10'
plt.rcParams["mathtext.fontset"] = 'cm'
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
import numpy as np
import xarray as xr
import metpy
from metpy.units import units

In [ ]:
from importlib import reload
import extract_forcing

In [ ]:
reload(extract_forcing)
from extract_forcing import *

In [ ]:
h = 30. # actual surface elevation of ENA site in m
lat = 39.0916 # deg N
lon = -28.0257 # deg E
forc_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/forcing'

In [ ]:
time_merra2 = np.arange(0.5, 23.7, 1.0)
merra2_l = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.20180207.nc4')
# merra2_l_ena = merra2_l.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.20180207.nc4')
merra2_sfc2 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.20180207.nc4')
merra2_sfc0 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_101.const_2d_asm_Nx.00000000.nc4')

In [ ]:
# t1 = np.datetime64('2018-02-07T00:00:00')
# t2 = np.datetime64('2018-02-08T00:00:00')
t1 = '2018-02-07T00:00:00'
t2 = '2018-02-08T00:00:00'
dx = 0.25
toy, sst, shf, lhf = extract_sfc_fluxes_merra2(merra2_sfc, merra2_sfc2, t1, t2, lat, lon, dx)

In [ ]:
sst

In [ ]:
frac_land = merra2_sfc0.FRLAND.loc[:, lat-dx:lat+dx, lon-dx:lon+dx]

In [ ]:
frac_land

In [ ]:
# z.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
# p.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
# d.w.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
# d.u.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
# d.v.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
# ta.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
# ma.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
# ug.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
# vg.loc[t1:t2,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx],
zs, z, p, w, u, v, ta, ma, ug, vg = extract_forcing_merra2(merra2_l, t1, t2, lat, lon, dx)

In [ ]:
zs[0,0,0]

In [ ]:
dz = 25. # m
zout = np.arange(dz*0.5, 5000.+ dz*0.1, dz)
wi, ui, vi, tai, mai, ugi, vgi = interpolate_forcing_merra2(zout, z, w, u, v, ta, ma, ug, vg, zs)

In [ ]:
tstamp = '2018-02-07T12:00:00'
zp = z.sel(time=tstamp, longitude=lon+360., latitude=lat, method='nearest')
up = u.sel(time=tstamp, longitude=lon+360., latitude=lat, method='nearest')
vp = v.sel(time=tstamp, longitude=lon+360., latitude=lat, method='nearest')
ugp = ug.sel(time=tstamp, longitude=lon+360., latitude=lat, method='nearest')
vgp = vg.sel(time=tstamp, longitude=lon+360., latitude=lat, method='nearest')

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
zt = 5
lim = zp < zt*1.0e3*units.meter
zlim = np.compress(lim, zp)
upp = np.compress(lim, up)
vpp = np.compress(lim, vp)
ugpp = np.compress(lim, ugp)
vgpp = np.compress(lim, vgp)
ax1.plot(upp, zlim*1.0e-3, color='black', label='u')
ax1.plot(ugpp, zlim*1.0e-3, color='blue', label='ug')
ax2.plot(vpp, zlim*1.0e-3, color='black', label='v')
ax2.plot(vgpp, zlim*1.0e-3, color='blue', label='vg')
# ax2.plot(np.sqrt(upp.values**2+vpp.values**2), zlim*1.0e-3, color='black', label='v')
# ax2.plot(np.sqrt(ugpp.values**2+vgpp.values**2), zlim*1.0e-3, color='blue', label='vg')
ax1.set_ylim(0, zt)
ax1.set_ylabel('Altitude (km)')
ax1.set_xlabel(r' $u$ (m/s)')
ax1.set_title(tstamp)
ax1.legend(fontsize=12)
ax2.set_ylim(0, zt)
ax2.set_ylabel('')
ax2.set_xlabel(r' $v$ (m/s)')
ax2.legend(fontsize=12)
ax2.set_title(tstamp)
plt.show()